In [294]:
import math
from datetime import datetime
import random
# performing linear algebra 
import numpy as np 

# data processing 
import pandas as pd 

# visualisation 
import matplotlib.pyplot as plt 

# logistic regression
from sklearn import linear_model 

# To split train and test set
from sklearn.model_selection import train_test_split 
# Confusion Matrix
from sklearn.metrics import confusion_matrix
from sklearn import svm

#    Prepare Dataset
   

## Fetch  data set 

In [295]:
data = pd.read_csv(".\\Data\\train.csv") 

#print (data)

# Handle data imbalance for Incorrect

In [296]:
incorrectLabeledData = data.loc[data['label'] == "incorrect"]
#incorrectLabeledData
incorrectLabeledData = incorrectLabeledData.reset_index(drop=True)
#print (incorrectLabeledData)

correctLabeledData = (data.loc[data['label'] == "correct"])

correctLabeledDataLength = correctLabeledData[correctLabeledData.columns[0]].count()
print(correctLabeledDataLength)
moreRowsToBalance = correctLabeledDataLength - incorrectLabeledData[incorrectLabeledData.columns[0]].count()
print(moreRowsToBalance)

indices = []
for x in range(1,moreRowsToBalance):
    indexToAdd = random.randrange(0,1680)
    indices.append(indexToAdd)
#print (toBeAdded)
#print (indices)
toBeAdded = incorrectLabeledData.iloc[indices, :]
#print (toBeAdded)

data.append(toBeAdded)

15495
13814


,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834.0,56.0,0.000000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791.0,47.0,0.000000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087.0,80.0,0.000000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598.0,271.0,15.663800,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,82.30,correct
4,189128020,NaN,NaN,NaN,NaN,NaN,11/1/2019 3:34,11/1/2019 3:51,6.87441,79.8615,6.84478,79.9290,358.39,correct
5,189129552,10.5,3407.0,182.0,0.000000,112.0,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,1065.02,correct
6,189132829,10.5,1246.0,487.0,0.000000,133.0,11/1/2019 6:29,11/1/2019 6:49,6.84371,79.9051,6.85069,79.8624,266.62,correct
7,189135103,10.5,1333.0,295.0,17.198500,212.0,11/1/2019 6:50,11/1/2019 7:12,6.90760,79.9524,6.90634,79.9042,318.05,correct
8,189139296,10.5,360.0,80.0,4.664000,3.0,11/1/2019 7:00,11/1/2019 7:06,7.26706,80.6064,7.27422,80.6124,100.32,correct
9,189138671,10.5,1539.0,588.0,33.986400,43.0,11/1/2019 7:02,11/1/2019 7:28,6.85137,79.9537,6.84779,79.9274,257.89,correct


### Calculate distance from longitude and latitude

In [297]:
R = 6373.0 # Radius of earth
distance_col = []
actual_duration_col = []
rep_time_diff_col = []

for index,row in data.iterrows():  
    #Calculate the distance 
    lat1 = math.radians(float(row["pick_lat"]))
    lon1 = math.radians(float(row["pick_lon"]))
    lat2 = math.radians(float(row["drop_lat"]))
    lon2 = math.radians(float(row["drop_lon"]))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    #Harversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    distance_col.append(distance)
    
    #Calculate actual duration
    start_time = datetime. strptime(row["pickup_time"], '%m/%d/%Y %H:%M')
    end_time = datetime. strptime(row["drop_time"], '%m/%d/%Y %H:%M')
    difference = end_time - start_time
    actual_duration_col.append(difference.total_seconds())
    #print(difference)

    #Calculate reported time difference
    rep_time_diff_col.append(difference.total_seconds() - (float(row["duration"]) + float(row["meter_waiting"])))
#print(distance_col)
#print (actual_duration_col)

### Adding new distance column to data set 

In [298]:
data['Distance'] = distance_col
data = data.drop(["pick_lat"], axis = 1) 
data = data.drop(["pick_lon"], axis = 1) 
data = data.drop(["drop_lat"], axis = 1) 
data = data.drop(["drop_lon"], axis = 1) 

data['ActualDuration'] = actual_duration_col
data= data.drop(["pickup_time"], axis = 1)
data= data.drop(["drop_time"], axis = 1)

data['ReportedDurationDiff'] = rep_time_diff_col
data = data.drop(["ActualDuration"], axis = 1)
data = data.drop(["duration"], axis = 1)
data = data.drop(["meter_waiting"], axis = 1)

data = data.drop(["additional_fare"], axis = 1)
data = data.drop(["meter_waiting_fare"], axis = 1)
data = data.drop(["tripid"], axis = 1)
data = data.drop(["meter_waiting_till_pickup"], axis = 1)
data = data.drop(["fare"], axis = 1)

### Calculate duration from start and end times 

In [299]:
#print (data)

In [300]:
data.label = [1 if each.strip() == "correct" else 0 for each in data.label] 

In [301]:
#Remove data rows that contain NaN values
data.dropna(subset=["Distance","ReportedDurationDiff"], inplace = True)
# Create train set
y = data.label.values 
x_data = data.drop(['label'], axis = 1) 

In [302]:
x_data

,Distance,ReportedDurationDiff
0,5.094369,-50.0
1,3.169052,-58.0
2,6.307375,-87.0
3,0.862217,-269.0
5,24.214638,-169.0
6,4.779123,-533.0
7,5.324215,-308.0
8,1.035627,-80.0
9,2.931635,-567.0
10,14.385516,-45.0


In [303]:
x = (x_data - np.min(x_data))/(np.max(x_data) - np.min(x_data)).values 

In [304]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.35, random_state = 42) 

#x_train = x_train.T 
#x_test = x_test.T 
#y_train = y_train.T 
#y_test = y_test.T 
print("X train : " , x_train.shape)
print("X test : " , x_test.shape)
print("Y train : " , y_train.shape)
print("Y test : " , y_test.shape)

X train :  (11033, 2)
X test :  (5941, 2)
Y train :  (11033,)
Y test :  (5941,)


## SVM

In [305]:
#fitting the model to the training set
#clf = svm.SVC(gamma='scale')
classes = ["correct","incorrect"]
clf = svm.SVC(kernel='linear')
clf.fit(x_train, y_train)


SVC(kernel='linear')

In [306]:
y_pred = clf.predict(x_test)
#con = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
true_pos = con[0][0]
false_neg = con[0][1]
false_pos = con[1][0]
true_neg = con[1][1]

accuracy = (tp + tn ) / (tp + tn + fp + fn)
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1_score = 2*precision*recall/(precision + recall)

print ("Accuracy : {}".format(accuracy) )
print ("Sensitivity : {}".format(sensitivity) )
print ("Specificity : {}".format(specificity) )
print ("Precision : {}".format(precision) )
print ("Recall : {}".format(recall) )
print ("F1 score : {}".format(f1_score) )
print(y_pred)

Accuracy : 0.9067497054367951
Sensitivity : 0.0037735849056603774
Specificity : 0.9998151229432427
Precision : 0.9068710003368138
Recall : 0.999814333457111
F1 score : 0.9510773578240904
[1 1 1 ... 1 1 1]


# Predicting to given Test data set

## Transform dataset

In [307]:
testData = pd.read_csv(".\\Data\\test.csv")
outData = pd.DataFrame()

outData["tripid"] = testData["tripid"]
R = 6373.0 # Radius of earth
distance_col = []
actual_duration_col = []
rep_time_diff_col = []

for index,row in testData.iterrows():  
    #Calculate the distance 
    lat1 = math.radians(float(row["pick_lat"]))
    lon1 = math.radians(float(row["pick_lon"]))
    lat2 = math.radians(float(row["drop_lat"]))
    lon2 = math.radians(float(row["drop_lon"]))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    #Harversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    distance_col.append(distance)
    
    #Calculate actual duration
    start_time = datetime. strptime(row["pickup_time"], '%m/%d/%Y %H:%M')
    end_time = datetime. strptime(row["drop_time"], '%m/%d/%Y %H:%M')
    difference = end_time - start_time
    actual_duration_col.append(difference.total_seconds())
    #print(difference)

    #Calculate reported time difference
    rep_time_diff_col.append(difference.total_seconds() - (float(row["duration"]) + float(row["meter_waiting"])))
#print(distance_col)
#print (actual_duration_col)

testData['Distance'] = distance_col
testData = testData.drop(["pick_lat"], axis = 1) 
testData = testData.drop(["pick_lon"], axis = 1) 
testData = testData.drop(["drop_lat"], axis = 1) 
testData = testData.drop(["drop_lon"], axis = 1) 

testData['ActualDuration'] = actual_duration_col
testData= testData.drop(["pickup_time"], axis = 1)
testData= testData.drop(["drop_time"], axis = 1)

testData['ReportedDurationDiff'] = rep_time_diff_col
testData = testData.drop(["ActualDuration"], axis = 1)
testData = testData.drop(["duration"], axis = 1)
testData = testData.drop(["meter_waiting"], axis = 1)

testData = testData.drop(["additional_fare"], axis = 1)
testData = testData.drop(["meter_waiting_fare"], axis = 1)
testData = testData.drop(["tripid"], axis = 1)
testData = testData.drop(["meter_waiting_till_pickup"], axis = 1)
testData = testData.drop(["fare"], axis = 1)

In [308]:
#print(testData)

In [309]:
y_pred = clf.predict(testData)

In [310]:
print(y_pred)

[0 0 0 ... 0 0 0]


# Create an Excel file

In [311]:

outData["prediction"] = y_pred
    

In [312]:
outData.to_csv("ExportedResults.csv",index=False)